In [ ]:
# ============================================================
# e1 Cluster Detection Using Random Forest (R Pipeline)
# ============================================================

library(randomForest)
library(caret)
library(pROC)
set.seed(486)

# ---------------------------
# Load data
# ---------------------------
library(readr)
csv_path <- "/kaggle/input/original-training-data/Original training DB e1 positive.csv"
df <- read_csv(csv_path, show_col_types = FALSE)

label_col <- "Label"
df[[label_col]] <- factor(as.character(df[[label_col]]), levels=c("0","1"))

In [54]:
# ---------------------------
# Audit of Original Database
# ---------------------------

# Dimensions
cat("Number of samples:", nrow(df), "\n")
cat("Number of features (excluding label):", ncol(df) - 1, "\n")

# Class distribution
cat("\nClass distribution:\n")
print(table(df[[label_col]]))

# Check class balance
class_counts <- table(df[[label_col]])
minority_ratio <- min(class_counts) / sum(class_counts)
cat(sprintf("\nMinority class ratio: %.4f\n", minority_ratio))

# Check for missing values
cat("\nMissing values per column:\n")
print(colSums(is.na(df)))

# Data types of features
cat("\nFeature types:\n")
print(sapply(df, class))

# Ratio of samples to features
ratio <- nrow(df) / (ncol(df) - 1)
cat(sprintf("\nSamples-to-features ratio: %.2f\n", ratio))

Number of samples: 871 
Number of features (excluding label): 608 

Class distribution:

  0   1 
572 299 

Minority class ratio: 0.3433

Missing values per column:
       GABRG2         CELF4         SRRM4        SLC1A3        ATP1A3 
            0             0             0             0             0 
       RBFOX3        GABRA4         NHSL1        GRAMD3        SEZ6L2 
            0             0             0             0             0 
         SV2A          PON2         SCN3B       DLX6AS1           SYP 
            0             0             0             0             0 
       ELAVL2        ADAM28          BEX1         DOCK8       FAM153C 
            0             0             0             0             0 
        PWRN1         KANK1         PTPRC        ADGRG1       APBB1IP 
            0             0             0             0             0 
     SYNPRAS1        INPP5D        CSF2RA         NRIP3       HEPACAM 
            0             0             0             

In [55]:
# ---------------------------
# Train / Verification Split
# ---------------------------
set.seed(486)
pick_one <- function(x, lab) sample(which(x == lab), 1)
i_pos <- pick_one(df[[label_col]], "1")
i_neg <- pick_one(df[[label_col]], "0")
verif_idx <- c(i_pos, i_neg)

verif_df <- df[verif_idx, ]
train_df <- df[-verif_idx, ]

In [ ]:
# ---------------------------
# 2) Define hyperparameter grid
# ---------------------------
p <- ncol(train_df) - 1   # exclude label
mtry_grid  <- unique(round(c(0.5*sqrt(p), sqrt(p), 2*sqrt(p))))
ntree_grid <- c(100, 500, 1000, 2000, 4000, 8000)
cut_grid   <- list(c(0.3,0.7), c(0.5,0.5), c(0.7,0.3))

results <- data.frame(ntree=integer(), mtry=integer(),
                      cutoff=I(list()), OOB=numeric())

best <- list(oob=Inf)


In [56]:
# ---------------------------
# Training / Verification DB Summaries
# ---------------------------
cat("\n--- Training DB Summary ---\n")
cat("Samples:", nrow(train_df), " | Features:", ncol(train_df) - 1, "\n")
print(table(train_df[[label_col]]))
cat(sprintf("Minority class ratio: %.4f\n",
            min(table(train_df[[label_col]])) / nrow(train_df)))

cat("\n--- Verification DB Summary ---\n")
cat("Samples:", nrow(verif_df), " | Features:", ncol(verif_df) - 1, "\n")
print(table(verif_df[[label_col]]))


--- Training DB Summary ---
Samples: 869  | Features: 608 

  0   1 
571 298 
Minority class ratio: 0.3429

--- Verification DB Summary ---
Samples: 2  | Features: 608 

0 1 
1 1 


In [57]:
# ---------------------------
# Hyperparameter grid
# ---------------------------
p <- ncol(train_df) - 1
mtry_grid  <- sort(unique(pmax(1L, round(c(0.5*sqrt(p), sqrt(p), 2*sqrt(p))))))

ntree_grid <- c(1000, 2000, 4000, 8000)

cut_grid   <- list(
  c(0.7, 0.3),
  c(0.5, 0.5),
  c(0.3, 0.7)
)

results <- data.frame(
  ntree = integer(),
  mtry  = integer(),
  cutoff_0 = numeric(),
  cutoff_1 = numeric(),
  OOB_Error = numeric(),
  Accuracy = numeric(),
  Precision = numeric(),
  Recall = numeric(),
  F1 = numeric()
)

best <- list(oob = Inf)

In [58]:
# ---------------------------
# Grid Search with OOB + Accuracy Metrics (OOB-based)
# ---------------------------
for (nt in ntree_grid) {
  for (mt in mtry_grid) {
    for (co in cut_grid) {
      fit <- randomForest(
        as.formula(paste(label_col, "~ .")),
        data       = train_df,
        ntree      = nt,
        mtry       = mt,
        cutoff     = co,
        importance = FALSE
      )
      oob_err <- tail(fit$err.rate[,"OOB"], 1)

      pred <- fit$predicted

      cm <- confusionMatrix(data = pred, reference = train_df[[label_col]], positive="1")
      acc  <- cm$overall["Accuracy"]
      prec <- cm$byClass["Precision"]
      rec  <- cm$byClass["Recall"]
      f1   <- cm$byClass["F1"]

      results <- rbind(results, data.frame(
        ntree     = nt,
        mtry      = mt,
        cutoff_0  = co[1],
        cutoff_1  = co[2],
        OOB_Error = as.numeric(oob_err),
        Accuracy  = as.numeric(acc),
        Precision = as.numeric(prec),
        Recall    = as.numeric(rec),
        F1        = as.numeric(f1)
      ))

      if (oob_err < best$oob) {
        best <- list(model = fit, ntree = nt, mtry = mt, cutoff = co, oob = oob_err)
      }
    }
  }
}

cat("\nBest Hyperparameters (true global minimum OOB):\n")
print(list(
  ntree  = best$ntree,
  mtry   = best$mtry,
  cutoff = best$cutoff,
  OOB    = best$oob
))


Best Hyperparameters (true global minimum OOB):
$ntree
[1] 1000

$mtry
[1] 12

$cutoff
[1] 0.5 0.5

$OOB
[1] 0.003452244



In [59]:
# ---------------------------
# Accuracy metrics for best model (OOB-based)
# ---------------------------
print(best$model)

# OOB confusion matrix (raw counts; rows = truth, cols = predicted)
conf_mat <- best$model$confusion[1:2, 1:2]
cat("\n--- OOB Confusion Matrix (counts) ---\n")
print(conf_mat)

TN <- conf_mat[1,1]; FP <- conf_mat[1,2]
FN <- conf_mat[2,1]; TP <- conf_mat[2,2]

accuracy  <- (TP + TN) / sum(conf_mat)
precision <- TP / (TP + FP)
recall    <- TP / (TP + FN)
f1        <- 2 * precision * recall / (precision + recall)

fmt5 <- function(x) format(round(x + 1e-15, 5), nsmall = 5)

cat("\n--- OOB Accuracy Metrics (5 decimals) ---\n")
cat(sprintf("Accuracy : %s\n", fmt5(accuracy)))
cat(sprintf("Precision: %s\n", fmt5(precision)))
cat(sprintf("Recall   : %s\n", fmt5(recall)))
cat(sprintf("F1 Score : %s\n", fmt5(f1)))

oob_err_final <- tail(best$model$err.rate[, "OOB"], 1)
cat(sprintf("OOB Error: %s\n", fmt5(oob_err_final)))


Call:
 randomForest(formula = as.formula(paste(label_col, "~ .")), data = train_df,      ntree = nt, mtry = mt, cutoff = co, importance = FALSE) 
               Type of random forest: classification
                     Number of trees: 1000
No. of variables tried at each split: 12

        OOB estimate of  error rate: 0.35%
Confusion matrix:
    0   1 class.error
0 568   3  0.00525394
1   0 298  0.00000000

--- OOB Confusion Matrix (counts) ---
    0   1
0 568   3
1   0 298

--- OOB Accuracy Metrics (5 decimals) ---
Accuracy : 0.99655
Precision: 0.99003
Recall   : 1.00000
F1 Score : 0.99499
OOB Error: 0.00345


In [ ]:
# ---------------------------
# Feature Ranking (final model retrained with ntree=8000)
# ---------------------------
final <- randomForest(
  as.formula(paste(label_col, "~ .")),
  data=train_df,
  ntree=8000,              # 8000 trees
  mtry=best$mtry,          # keep best mtry
  cutoff=best$cutoff,      # keep best cutoff
  importance=TRUE
)

imp <- importance(final, type=1)  # Mean Decrease Accuracy
imp_df <- data.frame(Feature=rownames(imp),
                     MDA=imp[,"MeanDecreaseAccuracy"])
imp_df <- imp_df[order(-imp_df$MDA), ]

cat("\nTop 10 Features (MDA, ntree=8000):\n")
print(head(imp_df, 10))



Top 10 Features (MDA, ntree=8000):
            Feature      MDA
TESPA1       TESPA1 32.59954
SLC17A7     SLC17A7 29.77607
LINC00507 LINC00507 28.05387
KCNIP1       KCNIP1 27.17743
ANKRD33B   ANKRD33B 26.98186
SLIT3.1     SLIT3.1 26.86996
SLIT3.2     SLIT3.2 26.65708
SLIT3         SLIT3 26.47532
SFTA1P       SFTA1P 26.30224
NRGN           NRGN 26.12710


In [40]:
saveRDS(final, "modelBEST.rds")

In [61]:
# ---------------------------
# Run Time Predictions on Verification DB
# ---------------------------
modelBEST <- readRDS("modelBEST.rds")
verif_probs <- predict(modelBEST, newdata=verif_df, type="prob")
verif_preds <- predict(modelBEST, newdata=verif_df, type="response")

out <- data.frame(
  ID = rownames(verif_df),
  TrueLabel = verif_df[[label_col]],
  Predicted = verif_preds,
  Prob0 = verif_probs[, "0"],
  Prob1 = verif_probs[, "1"],
  stringsAsFactors = FALSE
)

cat("\nVerification Results:\n")
print(out)


Verification Results:
  ID TrueLabel Predicted    Prob0    Prob1
1  1         1         1 0.028625 0.971375
2  2         0         0 0.944375 0.055625
